In [117]:
from datetime import date, datetime

import pandas as pd

from definitions import PICKLE_DIR
from api_utils import get_json_from_fpl_api
from df_utils import get_next_gameweek

In [29]:
json = get_json_from_fpl_api('http://fantasy.premierleague.com/api/bootstrap-static/')
json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [94]:
t = pd.DataFrame(json['events'])
t = t[['id','name', 'deadline_time', 'deadline_time_epoch', 'finished', 'data_checked', 'is_current', 'is_previous', 'is_next']]
t['deadline_time'] = t['deadline_time'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ"))
t['date_updated'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
t = t.set_index('id')

t.to_pickle(f"{PICKLE_DIR}/gw.pickle")

In [ ]:
next_gameweek = get_next_gameweek()
next_gameweek_fixtures_json = (get_json_from_fpl_api(f"http://fantasy.premierleague.com/api/fixtures/?event={next_gameweek}"))
next_gameweek_fixtures_df = pd.DataFrame(next_gameweek_fixtures_json)
next_gameweek_fixtures_df


In [180]:
df = next_gameweek_fixtures_df.merge(teams_df, left_on = 'team_h', right_on='fpl_team_id')\
  .rename(columns={'name': 'home_team', 'short_name': 'home_short_name'})\
    .merge(teams_df, left_on = 'team_a', right_on='fpl_team_id')\
    .rename(columns={'name': 'away_team', 'short_name': 'away_short_name'})\
      .sort_values(by='kickoff_time')

df['kickoff_time'] = df['kickoff_time'].apply( lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ'))
df['day'] = df['kickoff_time'].apply(lambda x: datetime.strftime(x, '%A'))
df['time'] = df['kickoff_time'].apply(lambda x: datetime.strftime(x, '%H:%M'))
df = df[['day', 'time', 'home_team', 'team_h_difficulty', 'away_team', 'team_a_difficulty']]
df


,day,time,home_team,team_h_difficulty,away_team,team_a_difficulty
0,Saturday,12:30,Man City,3,Newcastle,5
1,Saturday,15:00,Arsenal,2,Bournemouth,4
2,Saturday,15:00,Aston Villa,2,Crystal Palace,2
3,Saturday,15:00,Brighton,2,West Ham,3
4,Saturday,15:00,Chelsea,2,Leeds,4
5,Saturday,15:00,Wolves,3,Spurs,2
6,Saturday,17:30,Southampton,3,Leicester,2
7,Sunday,14:00,Nott'm Forest,2,Everton,2
8,Sunday,16:30,Liverpool,3,Man Utd,4
9,Monday,20:00,Brentford,2,Fulham,3
